# Setup

In [1]:
%matplotlib ipympl
import matplotlib, matplotlib.pyplot
matplotlib.pyplot.ioff()

import sys
sys.path.append("..")

from pathlib import Path
from analyze import Apps

cache_dir = Path("../data")

# Queries

## Number of custom components

In [2]:
with Apps.init(cache_dir) as apps:
    apps.where(lambda app: app.components.some(lambda component: component.type == "Custom")).select(
    {"Unique custom components": lambda app: app.components.filter(lambda component: component.type == "Custom")
        .map(lambda component: component[".tagName"])
        .filter(lambda tagName: tagName is not None)
        .unique()
        .length
    }) \
    .order_by(lambda app: (app.data["Unique custom components"],), reverse=True) \
    .table()

Env,Org,App,Unique custom components
tt02,dibk,mb-v3,10
tt02,dibk,ig-v3,10
tt02,dibk,fa-v3,9
tt02,dat,byggesak-samtykke-v3,6
tt02,dibk,innsending-planforslag,6
tt02,digdir,dan-data-app,1


Count: 6
